# Optimisation with `firedrake-adjoint`

*This example is modified from the equivalent [`dolfin-adjoint` demo](http://www.dolfin-adjoint.org/en/latest/documentation/stokes-bc-control/stokes-bc-control.html)*


In this example, we will look at how to use `firedrake-adjoint` to optimise for strong (Dirichlet) conditions in a steady problem.  `firedrake-adjoint` is a thin compatibility layer for the [`dolfin-adjoint` package](http://www.dolfin-adjoint.org/en/latest/), a python package to **automatically derive the discrete adjoint and tangent linear models** of forward problems written using Firedrake.

## Installing necessary dependencies
We will need to install `firedrake-adjoint` to run this example.  To do so, run:

```sh
firedrake-update --adjoint
```

In the activated firedrake virtualenv.

For the minimisation, we will need `scipy`, which is installed with

```sh
pip install scipy
```

In the activated firedrake virtualenv.

As usual, we begin with some notebook magic (so that plots appear nicely) and importing firedrake.

In [ ]:
%matplotlib notebook
from firedrake import *

The next step is new, we now import the `firedrake_adjoint` package, which *overrides* much of the Firedrake interface so that an *annotated tape* for adjoint calculations can be built automatically.

In [ ]:
from firedrake_adjoint import *

## Problem setup
Now we will set up the problem.  We consider minimising the compliance:

$$
\min_{g, u, p} \ \frac{1}{2}\int_{\Omega} \nabla u \cdot \nabla u\,\text{d}x +  \frac{\alpha}{2} \int_{\Gamma_{\textrm{in}}} g^2\,\text{d}s
$$

subject to the Stokes equations
$$
\begin{split}-\nu \Delta u + \nabla p &= 0  \quad \text{in $\Omega$} \\
                  \nabla \cdot u &= 0  \quad \text{in $\Omega$}  
                  \end{split}
$$

and Dirichlet conditions

$$
\begin{split}
u &= g  \quad \text{on $\Gamma_\text{circ}$} \\
u &= f  \quad \text{on $\Gamma_\text{in}$} \\
u &= 0  \quad \text{on $\Gamma_\text{top} \cup \Gamma_\text{bottom}$} \\
p &= 0  \quad \text{on $\Gamma_\text{out}$}. \\
\end{split}
$$

Here, $u$ and $p$ are unknown velocity and pressure, $f$ is a prescribed inflow, $g$ is the control variable that we will optimise for and $\alpha$ is a regularisation parameter.  This corresponds physically to minimising the loss of energy as heat by controlling the in/outflow on $\Gamma_\text{circ}$.  The regularisation parameter penalises too many non-zero control values.

This problem setup requires a mesh that is more complex than the built in ones Firedrake provides.  Instead, it was created with [Gmsh](http://gmsh.info).  It is loaded by using the `Mesh` constructor, passing the filename of the mesh in question.

In [ ]:
mesh = Mesh("stokes-control.msh")

Now we'll take a look at the mesh.  Since we will need to know which mesh markers correspond to which parts of the boundary, we've labelled them here.  Normally you will know this because you told your mesh generator how to mark the boundaries.

In [ ]:
axes = plot(mesh);
axes.axis("off")
axes.annotate("$\Gamma_\mathrm{in}$ (1)", xy=(-5, 5), annotation_clip=False, fontsize=16)
axes.annotate("$\Gamma_\mathrm{out}$ (2)", xy=(31, 5), annotation_clip=False, fontsize=16)
axes.annotate("$\Gamma_\mathrm{top}$ (3)", xy=(15, 11), annotation_clip=False, fontsize=16)
axes.annotate("$\Gamma_\mathrm{bottom}$ (5)", xy=(15, -2), annotation_clip=False, fontsize=16)
axes.annotate("$\Gamma_\mathrm{circ}$ (4)", xy=(8, 4.5), annotation_clip=False, fontsize=16);

## The forward problem

The forward problem should be familiar by now, we create a mixed function space for $u$ and $p$ and set up trial and test functions.  We specify a parabolic velocity at the inflow, and no-slip (zero velocity) conditions on the side walls.  The zero-pressure outflow condition is enforced weakly.

In [ ]:
V = VectorFunctionSpace(mesh, "CG", 2)
Q = FunctionSpace(mesh, "CG", 1)
W = V*Q

v, q = TestFunctions(W)
u, p = TrialFunctions(W)

nu = Constant(1)     # Viscosity coefficient

x, y = SpatialCoordinate(mesh)
u_inflow = as_vector([y*(10-y)/25.0, 0])

noslip = DirichletBC(W.sub(0), (0, 0), (3, 5))
inflow = DirichletBC(W.sub(0), interpolate(u_inflow, V), 1)
bcs = [inflow, noslip]

The variational formulation is slightly more complex.  At present, `firedrake-adjoint` cannot optimise boundary condition values in a `DirichletBC` object.  As a workaround, we therefore enforce the strong (Dirichlet) condition on the interior circle boundary using Nitsche's method.

We start with the normal weak formulation for Stokes.  Find $(u, p) \in V\times Q$ such that:

$$
\begin{align}
\int_\Omega \nu \nabla u : \nabla v\,\text{d}x
- \int_\Omega p \nabla \cdot v\,\text{d}x 
- \int_{\Gamma_{\text{circ}}} \nu (\nabla u \cdot n) \cdot v\,\text{d}s 
+ \int_{\Gamma_{\text{circ}}} p n \cdot v\,\text{d}s &= 0 \quad \forall v \in V\\
- \int_\Omega q \nabla \cdot u\,\text{d}x &= 0 \quad \forall q \in Q.
\end{align}
$$

Note how that the surface terms on $\Gamma_\text{circ}$ have not been dropped, since there is no DirichletBC constraining the values.

To enforce the boundary condition, we symmetrise the operator by adding new boundary terms plus the Nitsche term $\frac{\gamma}{h}\nu \int_{\Gamma_\text{circ}} u \cdot v\,\text{d}s$.  We then add the same terms to the right hand side, replacing the unknown $u$ by the prescribed boundary value $g$.  Here, $\gamma$ is the Nitsche parameter (typically chosen to be 10) and $h$ the mesh element size (available through the symbolic object `CellSize(mesh)`).

Finally, we arrive at the full formulation.  Find $(u, p) \in V\times Q$ such that:

$$
\begin{align}
\int_\Omega \nu \nabla u : \nabla v\,\text{d}x
- \int_\Omega p \nabla \cdot v\,\text{d}x \\
- \int_{\Gamma_{\text{circ}}} \nu (\nabla u \cdot n) \cdot v\,\text{d}s 
+ \int_{\Gamma_{\text{circ}}} p n \cdot v\,\text{d}s \\
- \int_{\Gamma_{\text{circ}}} \nu (\nabla v \cdot n) \cdot u\,\text{d}s
+ \int_{\Gamma_{\text{circ}}} \frac{\gamma}{h} \nu u \cdot v\,\text{d}s
&= -\int_{\Gamma_{\text{circ}}}\nu (\nabla v \cdot n) \cdot g\,\text{d}s 
+ \int_{\Gamma_{\text{circ}}} \frac{\gamma}{h} \nu g \cdot v\,\text{d}s\quad \forall v \in V\\
- \int_\Omega q \nabla \cdot u\,\text{d}x 
+ \int_{\Gamma_{\text{circ}}} qn\cdot u\,\text{d}s 
&= \int_{\Gamma_{\text{circ}}} qn\cdot g\,\text{d}s \quad \forall q \in Q.
\end{align}
$$

Let's go ahead and implement this.  First we define the control `Function` for the boundary values we wish to optimise for, along with the Nitsche parameter and a facet normal and cell size object.

In [ ]:
g = Function(V, name="Control")
gamma = Constant(10)
n = FacetNormal(mesh)
h = CellSize(mesh)

Now we define the bilinear and linear forms.  The input mesh had mesh markers for the different parts of the walls, and marker 4 corresponds to the facets on the circle boundary, so we use `ds(4)` to integrate over these pieces.

In [ ]:
a = (nu*inner(grad(u), grad(v))*dx
     - nu*inner(grad(u)*n, v)*ds(4)
     - nu*inner(grad(v)*n, u)*ds(4)
     + gamma/h*nu*inner(u, v)*ds(4)
     - inner(p, div(v))*dx
     + inner(p*n, v)*ds(4)
     - inner(q, div(u))*dx
     + inner(q*n, u)*ds(4)
     )
L = (- nu*inner(grad(v)*n, g)*ds(4)
     + gamma/h*nu*inner(g, v)*ds(4)
     + inner(q*n, g)*ds(4)
     )

Now let's solve the forward problem so that `firedrake-adjoint` annotates it.  We'll also take a look at the solution.

In [ ]:
w = Function(W)
solve(a == L, w, bcs=bcs, solver_parameters={"pc_type": "lu", "mat_type": "aij", 
                                             "pc_factor_mat_solver_package": "mumps"})

In [ ]:
u_init, p_init = w.split()
axes = plot(u_init);
axes.set_aspect("equal")
axes.set_title("Velocity")
axes = plot(p_init)
axes.set_aspect("equal")
axes.set_title("Pressure");

## The optimisation problem

Now we come to the optimisation problem.  We first define the functional we wish to minimise.  Then we specify a control variable, and produce a "reduced" functional (the evaluation of the functional at a given control value).  We then minimise (or minimi**z**e, due to American spellings) the functional, producing an optimised control value.  See [the dolfin-adjoint documentation](http://www.dolfin-adjoint.org/en/latest/documentation/optimisation.html#the-reduced-functional) for more details on reduced functionals.

In [ ]:
u, p = split(w)
alpha = Constant(10)

J = Functional(1./2*inner(grad(u), grad(u))*dx + alpha/2*inner(g, g)*ds(4))
m = Control(g)
Jhat = ReducedFunctional(J, m)
g_opt = minimise(Jhat)

Now let's take a look at the optimised control.  The initial boundary condition on the circle was no slip.  We see now that the optimised boundary condition has a significant outflow in the hole.

In [ ]:
axes = plot(g_opt, scale=3)
axes.set_aspect("equal")
axes.set_title("Optimised boundary value");

Let's compare the difference in the initial and final values of the functional.  This is done by calling the `ReducedFunctional` object with the control at the requested value.

In [ ]:
Jhat(g), Jhat(g_opt)

To see the optimised flow field, we solve the same problem again, only with the new (optimised) value for the boundary data on $\Gamma_\text{circ}$.  This time we're not interested in annotating the solve, so we tell `firedrake-adjoint` to ignore it by passing `annotate=False`.

In [ ]:
g.assign(g_opt)
w_opt = Function(W)
solve(a == L, w_opt, bcs=bcs, solver_parameters={"pc_type": "lu", "mat_type": "aij", 
                                                 "pc_factor_mat_solver_package": "mumps"},
      annotate=False)

u_opt, p_opt = w_opt.split()
axes = plot(u_opt)
axes.set_aspect("equal")
axes.set_title("Optimised velocity");
axes = plot(p_opt)
axes.set_aspect("equal")
axes.set_title("Optimised pressure");

Now that we are done optimising this problem, and so that the recorded annotations do not leak into the exercise, we tell `firedrake-adjoint` to *forget* all of the entries on its tape.

In [ ]:
adj_reset()

# Exercise

Modify the problem so that the boundary conditions are as follows:

$$
\begin{split}
u &= (1, 0) \quad &\text{on $\Gamma_\text{top}$}\\
u &= (0, 0) \quad&\text{on $\Gamma_\text{in}$}\\
u &= g \quad&\text{on $\Gamma_\text{out} \cup \Gamma_\text{bottom}$}\\
p &= 0 \quad&\text{on $\Gamma_\text{circ}$}
\end{split}
$$

and again optimise for $g$ in the same manner.

The change in the velocity and pressure fields will be less obvious than in the previous example, to verify that there is a difference try plotting the difference between the initial field for $u$ and the optimised field for $u$.

- Hint 1: the `ds` integral can take a `tuple` of markers: e.g. `ds((2, 5))`.
- Hint 2: to compute the pointwise difference between two fields in the same function space, write: `assemble(field_a - field_b)`.
